In [1]:
from share import *
import config

import cv2
import einops
import numpy as np
import torch
import random
import os
import json

from pytorch_lightning import seed_everything
from annotator.util import resize_image, HWC3
from annotator.midas import MidasDetector
from cldm.model import create_model, load_state_dict
from cldm.ddim_hacked import DDIMSampler
from PIL import Image
from tqdm import tqdm



apply_midas = MidasDetector()

model = create_model('./models/cldm_v21.yaml').cpu()
model.load_state_dict(load_state_dict('./lightning_logs/version_1/checkpoints/epoch=45-step=205620.ckpt', location='cuda'))
model = model.cuda()
ddim_sampler = DDIMSampler(model)


def process(input_image, prompt, a_prompt, n_prompt, num_samples, image_resolution, detect_resolution, ddim_steps, guess_mode, strength, scale, seed, eta, bg_threshold):
    with torch.no_grad():
        input_image = HWC3(input_image) # 이미지 전처리
        
       # _, detected_map = apply_midas(resize_image(input_image, detect_resolution), bg_th=bg_threshold)
       # detected_map = HWC3(input_image)  
        img = resize_image(input_image, image_resolution)
        H, W, C = img.shape

       # detected_map = cv2.resize(detected_map, (W, H), interpolation=cv2.INTER_LINEAR) 
        input_image = cv2.resize(input_image, (W, H), interpolation=cv2.INTER_LINEAR) 
    
    #   input_image = cv2.resize(detected_map, (W, H), interpolation=cv2.INTER_LINEAR) 

       # control = torch.from_numpy(detected_map[:, :, ::-1].copy()).float().cuda() / 255.0
        control = torch.from_numpy(input_image[:, :, ::-1].copy()).float().cuda() / 255.0
        control = torch.stack([control for _ in range(num_samples)], dim=0)
        control = einops.rearrange(control, 'b h w c -> b c h w').clone()

        if seed == -1:
            seed = random.randint(0, 65535)
        seed_everything(seed)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=False)

        cond = {"c_concat": [control], "c_crossattn": [model.get_learned_conditioning([prompt + ', ' + a_prompt] * num_samples)]}
        un_cond = {"c_concat": None if guess_mode else [control], "c_crossattn": [model.get_learned_conditioning([n_prompt] * num_samples)]}
        shape = (4, H // 8, W // 8)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=True)

        model.control_scales = [strength * (0.825 ** float(12 - i)) for i in range(13)] if guess_mode else ([strength] * 13)  # Magic number. IDK why. Perhaps because 0.825**12<0.01 but 0.826**12>0.01
        samples, intermediates = ddim_sampler.sample(ddim_steps, num_samples,
                                                     shape, cond, verbose=False, eta=eta,
                                                     unconditional_guidance_scale=scale,
                                                     unconditional_conditioning=un_cond)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=False)

        x_samples = model.decode_first_stage(samples)
        x_samples = (einops.rearrange(x_samples, 'b c h w -> b h w c') * 127.5 + 127.5).cpu().numpy().clip(0, 255).astype(np.uint8)

        results = [x_samples[i] for i in range(num_samples)]
    return results

def load_image_to_ndarray(file_path):
    try:
        pil_image = Image.open(file_path)
        image_array = np.array(pil_image)
        return image_array
    except IOError:
        print(f"Unable to load the image from file: {file_path}")
        return None
    
# Parameters
a_prompt = 'best quality, extremely detailed'
n_prompt = 'longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'
num_samples = 1
image_resolution = 512
detect_resolution = 384
ddim_steps = 50
guess_mode = False
strength = 1.0
scale = 9.0
seed = random.randint(1000000, 9999999)
eta = 0.0
bg_threshold = 0.4

# Inputs
relative_path = '../fire_gen/fire_and_masked_images_30_real_new/train/masked/'
file_list = ['S3-N1407MF01125.jpg', 'S3-N1411MF00607.jpg', 'S3-N1409MF01699.jpg', 'S3-N0886MF00021.jpg', 'S3-N1411MF00062.jpg', 'S3-N0840MF01423.jpg', 'S3-N0903MF01373.jpg']
image_file_list = sorted([relative_path + i for i in file_list])

prompt_list = ['A house is situated with an outdoor fire burning on firewood in front of it.',
               'Near a house with white and black walls, there is a small orange flame flickering in the grass.',
               'In close proximity to the grass, there stands an old house with a combination of gray and white colors.,',
               'Some trees and bushes form the background as a large pile of dirt sits on the side of a road.',
               'Surrounded by trees, a small gray cabin with a cozy fire burning inside provides a picturesque scene.',
               'Next to the wall of a brick house, a small fire is smoldering, emitting a faint glow.',
               'A hillside is visible in the background of the construction site.'
               ]

output_dir = 'inference_fire'
os.makedirs(output_dir, exist_ok = True)

for i in tqdm(range(len(image_file_list))):
    image_file_path = image_file_list[i]
    input_image = load_image_to_ndarray(image_file_path)
    prompt = prompt_list[i]

    infer = process(input_image, prompt, a_prompt, n_prompt, num_samples, image_resolution, detect_resolution, ddim_steps, guess_mode, strength, scale, seed, eta, bg_threshold)

    pil_image = Image.fromarray(infer[0])

    image_file_name = image_file_path
    pil_image.save(os.path.join(output_dir, image_file_path[-19:]))

2023-07-20 23:02:00.439553: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


logging improved.
ControlLDM: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 20 heads.
Setting up MemoryEfficientCrossAttention. 

  0%|          | 0/7 [00:00<?, ?it/s]Global seed set to 1386433


Data shape for DDIM sampling is (1, 4, 64, 112), eta 0.0
Running DDIM Sampling with 50 timesteps


 14%|█▍        | 1/7 [00:19<01:58, 19.75s/it]Global seed set to 1386433


...

 71%|███████▏  | 5/7 [01:37<00:38, 19.44s/it]Global seed set to 1386433


Data shape for DDIM sampling is (1, 4, 64, 112), eta 0.0
Running DDIM Sampling with 50 timesteps


 86%|████████▌ | 6/7 [01:56<00:19, 19.48s/it]Global seed set to 1386433


Data shape for DDIM sampling is (1, 4, 64, 112), eta 0.0
Running DDIM Sampling with 50 timesteps


100%|██████████| 7/7 [02:16<00:00, 19.47s/it]
